# Redes densas: load_wine

In [1]:
import numpy as np
import time
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
np.random.seed(1234)

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling1D, Conv1D
from tensorflow.keras.metrics import MSE
from tensorflow.keras.optimizers.legacy import Adadelta
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Dataset


In [3]:
wines = load_wine()

In [20]:
wines['feature_names']

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [19]:
df_wine = pd.DataFrame( np.c_[wines['data'], wines['target']], columns=np.append(wines['feature_names'], ['target']) )
df_wine

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2.0


In [6]:
#sns.pairplot(df_wine, hue = 'target', vars = wines['feature_names'])

In [7]:
n_classes = 3
lr = 1.0
epochs = 30
batch_size = 128

## Consignas

* Dividir el dataset usando train_test_split()
* Pasar las etiquetas a vectores en formato *one-hot encoding* con la funcion _to_categorical()_
* Crear un modelo de **redes densas** y resolver el problema. Graficar las curvas de Accuracy y MSE (u otra metrica a eleccion) en funcion de las _epochs_.
* ¿Como cambia el resultado si aplico esta funcion de pre-procesamiento antes del entrenamiento? **IMPORTANTE:** Aplicar sobre los datos de *train* y *test*.

In [8]:
scaler = StandardScaler()
# scaler.fit(wines.data)
#datos = scaler.fit_transform(wines.data)

In [9]:
# Crear x_train, x_test, y_train, y_test
# 1 - fitear el scaler con x_train.
# 2 - transformar x_train con el scaler.
# 3 - transformar x_test con el scaler (sin volver a fitear).

In [10]:
X_train, X_test, y_train, y_test = train_test_split(wines.data, wines.target, test_size=0.25, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes) #one-hot encoding

In [11]:
input_layer = Input(shape = (X_train.shape[1:]))
dense_layer_1 = Dense(128, activation = 'relu')(input_layer)
dense_layer_2 = Dense(64, activation = 'relu')(dense_layer_1)
dropout =  Dropout(0.2)(dense_layer_2)
dense_layer_3 = Dense(32, activation = 'relu')(dropout)
dense_layer_4 = Dense(16, activation = 'relu')(dense_layer_3)
dropout_2 = Dropout(0.2)(dense_layer_4)
output_layer = Dense(n_classes, activation = 'softmax')(dropout_2)
model_dense = Model(input_layer, output_layer)


In [12]:
Adadelta_optimizer = Adadelta(learning_rate=lr, rho=0.95)
model_dense.compile(optimizer=Adadelta_optimizer, loss='categorical_crossentropy', metrics=['acc', 'mse'])
model_dense.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 128)               1792      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0     

In [13]:
start_time = time.time()
history_dense = model_dense.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), shuffle=True, verbose=1)
end_time = time.time()
print('\nElapsed Dense Model training time: {:.5f} seconds'.format(end_time-start_time))

Epoch 1/30


2/2 [==============================] - 1s 227ms/step - loss: 1.1426 - acc: 0.3083 - mse: 0.2312 - val_loss: 1.0745 - val_acc: 0.4667 - val_mse: 0.2169
Epoch 2/30
2/2 [==============================] - 0s 33ms/step - loss: 1.0865 - acc: 0.3459 - mse: 0.2199 - val_loss: 1.0269 - val_acc: 0.6222 - val_mse: 0.2068
Epoch 3/30
2/2 [==============================] - 0s 30ms/step - loss: 1.0394 - acc: 0.4812 - mse: 0.2100 - val_loss: 0.9842 - val_acc: 0.7333 - val_mse: 0.1972
Epoch 4/30
2/2 [==============================] - 0s 30ms/step - loss: 0.9967 - acc: 0.6015 - mse: 0.2006 - val_loss: 0.9478 - val_acc: 0.7556 - val_mse: 0.1893
Epoch 5/30
2/2 [==============================] - 0s 29ms/step - loss: 0.9550 - acc: 0.6165 - mse: 0.1915 - val_loss: 0.9031 - val_acc: 0.7556 - val_mse: 0.1793
Epoch 6/30
2/2 [==============================] - 0s 32ms/step - loss: 0.9249 - acc: 0.6165 - mse: 0.1852 - val_loss: 0.8563 - val_acc: 0.8444 - val_mse: 0.1687
Epoch 7/30
2/2 [============================

* **EXTRA:** Resolver el problema anterior usando algunas capas ocultas convolucionales (Conv1D). Medir tiempos y comaparar resultados.

In [14]:
wines = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wines.data, wines.target, test_size=0.25, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes) #one-hot encoding

In [15]:
# Model
#---------------------------------------------------------------------#
input_layer = Input(shape=(X_train.shape[1], 1))
conv_1 = Conv1D(32, 3, activation='relu') (input_layer)
conv_2 = Conv1D(64, 3, activation='relu') (conv_1)
pool_1 = MaxPooling1D(pool_size=2) (conv_2)
dropout_1 = Dropout(0.25) (pool_1)
flatten_1 = Flatten() (dropout_1)
dense_1 = Dense(100, activation='relu') (flatten_1)
dropout_2 = Dropout(0.25) (dense_1)
output_layer = Dense(n_classes, activation='softmax') (dropout_2)
#---------------------------------------------------------------------#
model_conv = Model(input_layer, output_layer)

In [16]:
model_conv.compile(optimizer=Adadelta_optimizer, loss='categorical_crossentropy', metrics=['acc', 'mse'])
model_conv.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 13, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 11, 32)            128       
                                                                 
 conv1d_1 (Conv1D)           (None, 9, 64)             6208      
                                                                 
 max_pooling1d (MaxPooling1  (None, 4, 64)             0         
 D)                                                              
                                                                 
 dropout_2 (Dropout)         (None, 4, 64)             0         
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                           

Trainable params: 32339 (126.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
start_time = time.time()
history_conv = model_conv.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), shuffle=True, verbose=1)
end_time = time.time()
print('\nElapsed Dense Model training time: {:.5f} seconds'.format(end_time-start_time))

Epoch 1/30
2/2 [==============================] - 1s 209ms/step - loss: 1.1190 - acc: 0.3083 - mse: 0.2264 - val_loss: 1.0365 - val_acc: 0.5333 - val_mse: 0.2085
Epoch 2/30
2/2 [==============================] - 0s 32ms/step - loss: 1.0545 - acc: 0.4286 - mse: 0.2125 - val_loss: 0.9797 - val_acc: 0.6222 - val_mse: 0.1950
Epoch 3/30
2/2 [==============================] - 0s 29ms/step - loss: 1.0137 - acc: 0.4737 - mse: 0.2028 - val_loss: 0.9119 - val_acc: 0.8444 - val_mse: 0.1807
Epoch 4/30
2/2 [==============================] - 0s 32ms/step - loss: 0.9255 - acc: 0.6692 - mse: 0.1842 - val_loss: 0.8608 - val_acc: 0.8000 - val_mse: 0.1680
Epoch 5/30
2/2 [==============================] - 0s 30ms/step - loss: 0.8909 - acc: 0.6316 - mse: 0.1759 - val_loss: 0.8034 - val_acc: 0.8667 - val_mse: 0.1548
Epoch 6/30
2/2 [==============================] - 0s 30ms/step - loss: 0.8180 - acc: 0.7669 - mse: 0.1587 - val_loss: 0.7564 - val_acc: 0.8000 - val_mse: 0.1480
Epoch 7/30
2/2 [=================